# Checking NVIDIA Driver

Should be NVIDIA TESLA P100 or TESLA P4 so that we can complete training and prediction in one hour
if TESLA K80 is shown then deep learning model will take 10 hours just to finish training  

In [0]:
!nvidia-smi

# Importing the libraries

In [0]:
import os
import gc
import copy
import random
import sklearn
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import tensorflow as tf
from sklearn import metrics
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from IPython.display import display,HTML
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import losses, models, optimizers
from tensorflow.compat.v1.keras.layers import CuDNNLSTM 
from IPython.core.interactiveshell import InteractiveShell
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import categorical_crossentropy
from sklearn.model_selection import KFold, train_test_split, GroupKFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, Callback
from tensorflow.keras.layers import Dense, Dropout, Add, Conv1D, Flatten,BatchNormalization, Activation, AveragePooling1D,Bidirectional, Multiply,SpatialDropout1D
from tensorflow.keras.layers import GlobalAveragePooling1D, Lambda,Input, Concatenate, UpSampling1D, Multiply,MaxPooling1D,LSTM,TimeDistributed

In [0]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
InteractiveShell.ast_node_interactivity = 'all'
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (20,10)

# Helper Functions

In [0]:
#Feature Engineering Function
def get_features(df : pd.DataFrame,
                diff = False,
                shift_sizes = [1, 2, 3, 4, 5],
                add_pct_change = False,
                add_pct_change_lag = False) -> pd.DataFrame:


    df['group'] = df.groupby(df.index//4000).agg('ngroup').astype(np.int32)

    for shift_size in tqdm_notebook(shift_sizes,leave=False):    
        df['signal_shift_pos_'+str(shift_size)] = df.groupby('group')['signal'].shift(shift_size).bfill(0)
        df['signal_shift_neg_'+str(shift_size)] = df.groupby('group')['signal'].shift(-1*shift_size).ffill(0)
        if diff & shift_size == 1:
          df['signal_diff_pos_'+str(shift_size)] = df.groupby('group')['signal'].diff(shift_size).bfill(0)
          df['signal_diff_neg_'+str(shift_size)] = df.groupby('group')['signal'].diff(-1*shift_size).ffill(0)

    if add_pct_change:
        df['pct_change'] = df['signal'].pct_change()
        if add_pct_change_lag:
            for shift_size in shift_sizes:    
                df['pct_change_shift_pos_'+str(shift_size)] = df.groupby('group')['pct_change'].shift(shift_size).fillna(0)
                df['pct_change_shift_neg_'+str(shift_size)] = df.groupby('group')['pct_change'].shift(-1*shift_size).fillna(0)

    df['sig_power_1'] = df['signal']**2
    df['sig_power_2'] = df['signal']**2
    return df

#Calculating the Mean Absolute Deviation
def mean_abs_dev(val):
  return np.mean(np.absolute(val - np.mean(val)))


#Stretches the signal data(Time Series Data Augumentation)
def stretch(data, rate=1):
    input_length = data.shape[0]
    data = librosa.effects.time_stretch(data, rate)
    if len(data)>input_length:
        data = data[:input_length]
    else:
        data = np.pad(data, (0, max(0, input_length - len(data))), "constant")

    return data

In [0]:
#For Normalizing
def std_sc(df,columns):
  for x in columns:
    df[x] = (df[x] - df[x].mean())/df[x].std()
  return df

In [0]:
#Eval Metric For LightGBM
def MacroF1Metric(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.round(np.clip(preds, 0, 10)).astype(int)
    score = f1_score(labels, preds, average = 'macro')
    return ('MacroF1Metric', score, True)

In [0]:
#Custom Mish Activation Function
def mish(x):
	return tf.keras.layers.Lambda(lambda x: x*K.tanh(K.softplus(x)))(x)
get_custom_objects().update({'mish': Activation(mish)})

In [0]:
def get_one_sec_batch(df):
  df['one_sec_batch'] = df.groupby(df.index//10_000).agg('ngroup').astype(np.int32)
  return df

def get_mini_batch(df):
  df['mini_batch'] = df.groupby(df.index//5000).agg('ngroup').astype(np.int32)
  return df

def get_batches(df):
  df['batch'] =  df.groupby(df.index//50_000).agg('ngroup').astype(np.int16)
  return df

In [0]:
#Anonymous Functions
def reduce_mem_usage(df: pd.DataFrame,verbose: bool = True) -> pd.DataFrame:
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int8).min
                        and c_max < np.iinfo(np.int8).max):
                    df[col] = df[col].astype(np.int8)
                elif (c_min > np.iinfo(np.int16).min
                      and c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df


def show(df,col,mask='open_channels',alpha=None):
  for x in tqdm_notebook(np.sort(df[mask].unique())):
    _=plt.plot(df[df[mask] == x][col],label=x,alpha=alpha)
  _=plt.legend()


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

def submit(file):
  submission.to_csv(f'{file}.csv',index=False,float_format='%.4f')

In [0]:
#WaveNet Model
def WaveNetResidualConv1D(num_filters, kernel_size, stacked_layer):

    def build_residual_block(l_input):
        resid_input = l_input
        for dilation_rate in [2**i for i in range(stacked_layer)]:
            l_sigmoid_conv1d = Conv1D(
              num_filters, kernel_size, dilation_rate=dilation_rate,
              padding='same', activation='sigmoid')(l_input)
            l_tanh_conv1d = Conv1D(
             num_filters, kernel_size, dilation_rate=dilation_rate,
             padding='same', activation='mish')(l_input)
            l_input = Multiply()([l_sigmoid_conv1d, l_tanh_conv1d])
            l_input = Conv1D(num_filters, 1, padding='same')(l_input)
            resid_input = Add()([resid_input ,l_input])
        return resid_input
    return build_residual_block
def Classifier(shape_):
    num_filters_ = 16
    kernel_size_ = 3
    stacked_layers_ = [12, 8, 4, 1]
    l_input = Input(shape=(shape_))
    x = Conv1D(num_filters_, 1, padding='same')(l_input)
    x = WaveNetResidualConv1D(num_filters_, kernel_size_, stacked_layers_[0])(x)
    x = Conv1D(num_filters_*2, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*2, kernel_size_, stacked_layers_[1])(x)
    x = Conv1D(num_filters_*4, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*4, kernel_size_, stacked_layers_[2])(x)
    x = Conv1D(num_filters_*8, 1, padding='same')(x)
    x = WaveNetResidualConv1D(num_filters_*8, kernel_size_, stacked_layers_[3])(x)
    l_output = Dense(1, activation='linear')(x)
    model = models.Model(inputs=[l_input], outputs=[l_output])
    opt = Adam(lr=LR)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    return model

In [0]:
#Custom Learning Schedule
def lr_schedule(epoch):
    if epoch < 40:
        lr = LR
    elif epoch < 50:
        lr = LR / 2
    elif epoch < 60:
        lr = LR / 4
    elif epoch < 75:
        lr = LR / 6
    elif epoch < 85:
        lr = LR / 8
    elif epoch < 100:
        lr = LR / 10
    else:
        lr = LR / 12
    return lr

# Helper Classes

In [0]:
#Eval Metric for Deep Neural Network
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis = 2).reshape(-1)
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis = 2).reshape(-1)
        score = f1_score(self.targets, pred, average = 'macro')
        print(f'F1 Macro Score: {score:.5f}')

# Loading the data

Signal Data is Cleaned with the Kalman filter to remove the noise

In [0]:
train = pd.read_csv('train_clean_kalman.csv')
test = pd.read_csv('test_clean_kalman.csv')
submission = pd.read_csv('sample_submission.csv.zip')
test.reset_index(drop=True,inplace=True)

#### Sampling

In [0]:
train = get_one_sec_batch(train)
test = get_one_sec_batch(test)

In [0]:
train = get_mini_batch(train)
test = get_mini_batch(test)

In [0]:
train = get_batches(train)
test = get_batches(test)

Grouping the data according the number of states that can occur in the signal range

In [0]:
train_groups = {
    0:[(0,1000000)],
    1:[(1500000,2000000),(3500000,4000000)],
    2:[(2500000,3000000),(4000000,4500000)],
    3:[(1000000,1500000),(3000000,3500000)],
    4:[(2000000,2500000),(4500000,5000000)]
}

train['group'] = -1
for k,v in train_groups.items():
    for l,r in v:
        train.iloc[l:r,-1] = k


test['group'] = -1
test_groups = {
    0:[(0,100000),(300000,400000),(800000,900000),(1000000,2000000)],
    1:[(100000,200000),(900000,1000000)],
    2:[(200000,300000),(600000,700000)],
    3:[(400000,500000)],
    4:[(500000,600000),(700000,800000)]
}
for k,v in test_groups.items():
    for l,r in v:
        test.iloc[l:r,-1] = k

# Feature Engineering

In [0]:
train['has_10_states'] = np.where(train['group'] == 4,1,0)
test['has_10_states'] = np.where(test['group'] == 4,1,0)

min_mean_val = train[train['open_channels'] == 0]['signal'].mean()
train['signal'] = np.where(train['has_10_states'] == 1,train['signal'] - (min_mean_val),train['signal'])
test['signal'] = np.where(test['has_10_states'] == 1,test['signal'] - (min_mean_val),test['signal'])

train.drop(columns=['has_10_states'],inplace=True)
test.drop(columns=['has_10_states'],inplace=True)

In [0]:
train['batch_sig_mean'] = train.groupby('batch')['signal'].transform('mean')
train['batch_sig_std'] = train.groupby('batch')['signal'].transform('std')
train['batch_sig_max'] = train.groupby('batch')['signal'].transform('max')
train['batch_sig_min'] = train.groupby('batch')['signal'].transform('min')
train['batch_sig_max_min_ratio'] = train['batch_sig_max'] / train['batch_sig_min']
train['batch_sig_max_min_range'] = train['batch_sig_max'] - train['batch_sig_min']


test['batch_sig_mean'] = test.groupby('batch')['signal'].transform('mean')
test['batch_sig_std'] = test.groupby('batch')['signal'].transform('std')
test['batch_sig_max'] = test.groupby('batch')['signal'].transform('max')
test['batch_sig_min'] = test.groupby('batch')['signal'].transform('min')
test['batch_sig_max_min_ratio'] = test['batch_sig_max'] / test['batch_sig_min']
test['batch_sig_max_min_range'] = test['batch_sig_max'] - test['batch_sig_min']

In [0]:
train['mini_batch_sig_mean'] = train.groupby('mini_batch')['signal'].transform('mean')
train['mini_batch_sig_std'] = train.groupby('mini_batch')['signal'].transform('std')
train['mini_batch_sig_max'] = train.groupby('mini_batch')['signal'].transform('max')
train['mini_batch_sig_min'] = train.groupby('mini_batch')['signal'].transform('min')
train['mini_batch_sig_max_min_ratio'] = train['mini_batch_sig_max'] / train['mini_batch_sig_min']
train['mini_batch_sig_max_min_range'] = train['mini_batch_sig_max'] - train['mini_batch_sig_min']

test['mini_batch_sig_mean'] = test.groupby('mini_batch')['signal'].transform('mean')
test['mini_batch_sig_std'] = test.groupby('mini_batch')['signal'].transform('std')
test['mini_batch_sig_max'] = test.groupby('mini_batch')['signal'].transform('max')
test['mini_batch_sig_min'] = test.groupby('mini_batch')['signal'].transform('min')
test['mini_batch_sig_max_min_ratio'] = test['mini_batch_sig_max'] / test['mini_batch_sig_min']
test['mini_batch_sig_max_min_range'] = test['mini_batch_sig_max'] - test['mini_batch_sig_min']

In [0]:
train['one_sec_batch_sig_mean'] = train.groupby('one_sec_batch')['signal'].transform('mean')
train['one_sec_batch_sig_std'] = train.groupby('one_sec_batch')['signal'].transform('std')
train['one_sec_batch_sig_max'] = train.groupby('one_sec_batch')['signal'].transform('max')
train['one_sec_batch_sig_min'] = train.groupby('one_sec_batch')['signal'].transform('min')
train['one_sec_batch_sig_max_min_ratio'] = train['one_sec_batch_sig_max'] / train['one_sec_batch_sig_min']
train['one_sec_batch_sig_max_min_range'] = train['one_sec_batch_sig_max'] - train['one_sec_batch_sig_min']

test['one_sec_batch_sig_mean'] = test.groupby('one_sec_batch')['signal'].transform('mean')
test['one_sec_batch_sig_std'] = test.groupby('one_sec_batch')['signal'].transform('std')
test['one_sec_batch_sig_max'] = test.groupby('one_sec_batch')['signal'].transform('max')
test['one_sec_batch_sig_min'] = test.groupby('one_sec_batch')['signal'].transform('min')
test['one_sec_batch_sig_max_min_ratio'] = test['one_sec_batch_sig_max'] / test['one_sec_batch_sig_min']
test['one_sec_batch_sig_max_min_range'] = test['one_sec_batch_sig_max'] - test['one_sec_batch_sig_min']

In [0]:
cols = [x for x in train.columns[7:]]
for x in cols:
  train[f'{x}_msig'] = train[x] - train['signal']
  test[f'{x}_msig'] = test[x] - test['signal']

In [0]:
add_pct_change = True
diff = True
add_pct_change_lag = False
train = get_features(train,add_pct_change = add_pct_change,diff = diff,
                        add_pct_change_lag = add_pct_change_lag)
test = get_features(test,add_pct_change = add_pct_change,diff = diff,
                        add_pct_change_lag = add_pct_change_lag)

In [0]:
train['roll5_sig_q8'] = train['signal'].rolling(5,min_periods=1).quantile(0.8)
train['roll5_sig_q2'] = train['signal'].rolling(5,min_periods=1).quantile(0.2)
train['roll5_sig_q9'] = train['signal'].rolling(5,min_periods=1).quantile(0.9)
train['roll5_sig_q1'] = train['signal'].rolling(5,min_periods=1).quantile(0.1)

test['roll5_sig_q8'] = test['signal'].rolling(5,min_periods=1).quantile(0.8)
test['roll5_sig_q2'] = test['signal'].rolling(5,min_periods=1).quantile(0.2)
test['roll5_sig_q9'] = test['signal'].rolling(5,min_periods=1).quantile(0.9)
test['roll5_sig_q1'] = test['signal'].rolling(5,min_periods=1).quantile(0.1)

train['sig_IDR_range'] = train['roll5_sig_q8'] - train['roll5_sig_q2']
test['sig_IDR_range'] = test['roll5_sig_q8'] - test['roll5_sig_q2']

train['sig_abs_IDR_range'] = np.abs(train['roll5_sig_q8'] - train['roll5_sig_q2'])
test['sig_abs_IDR_range'] = np.abs(test['roll5_sig_q8'] - test['roll5_sig_q2'])

In [0]:
train['roll5_sig_std'] = train['signal'].rolling(5,min_periods=1).std()
train['roll5_sig_roll5_max_std'] = train['signal'].rolling(5,min_periods=1).std().rolling(5,min_periods=1).max()
test['roll5_sig_std'] = test['signal'].rolling(5,min_periods=1).std()
test['roll5_sig_roll5_max_std'] = test['signal'].rolling(5,min_periods=1).std().rolling(5,min_periods=1).max()

In [0]:
train['roll5_sig_std_IDR_range'] = train['roll5_sig_std'].rolling(5,min_periods=1).quantile(0.8) - train['roll5_sig_std'].rolling(5,min_periods=1).quantile(0.2)
test['roll5_sig_std_IDR_range'] = test['roll5_sig_std'].rolling(5,min_periods=1).quantile(0.8) - test['roll5_sig_std'].rolling(5,min_periods=1).quantile(0.2)

In [0]:
noise = np.random.normal(0,0.001,train.shape[0])
train['signal_with_sd_noise'] = train['signal'] + noise
noise = np.random.normal(0,0.001,test.shape[0])
test['signal_with_sd_noise'] = test['signal'] + noise[:test.shape[0]]

In [0]:
train['rev_sig'] = -1 * train['signal']
test['rev_sig'] = -1 * test['signal']

train['signal_stretch'] = stretch(train['signal'].values,0.8)
test['signal_stretch'] = stretch(test['signal'].values,0.8)

train['signal_wave_pitch'] = librosa.effects.pitch_shift(np.array(train['signal']),10,n_steps=-5)
test['signal_wave_pitch'] = librosa.effects.pitch_shift(np.array(test['signal']),10,n_steps=-5)

# Data Split

In [0]:
use_cols = [x for x in train.columns if x not in ['time', 'open_channels','batch', 'batch_slices', 'group','mini_batch','one_sec_batch']]
use_cols,len(use_cols)

(['signal',
  'batch_sig_mean',
  'batch_sig_std',
  'batch_sig_max',
  'batch_sig_min',
  'batch_sig_max_min_ratio',
  'batch_sig_max_min_range',
  'mini_batch_sig_mean',
  'mini_batch_sig_std',
  'mini_batch_sig_max',
  'mini_batch_sig_min',
  'mini_batch_sig_max_min_ratio',
  'mini_batch_sig_max_min_range',
  'one_sec_batch_sig_mean',
  'one_sec_batch_sig_std',
  'one_sec_batch_sig_max',
  'one_sec_batch_sig_min',
  'one_sec_batch_sig_max_min_ratio',
  'one_sec_batch_sig_max_min_range',
  'power',
  'batch_sig_mean_msig',
  'batch_sig_std_msig',
  'batch_sig_max_msig',
  'batch_sig_min_msig',
  'batch_sig_max_min_ratio_msig',
  'batch_sig_max_min_range_msig',
  'mini_batch_sig_mean_msig',
  'mini_batch_sig_std_msig',
  'mini_batch_sig_max_msig',
  'mini_batch_sig_min_msig',
  'mini_batch_sig_max_min_ratio_msig',
  'mini_batch_sig_max_min_range_msig',
  'one_sec_batch_sig_mean_msig',
  'one_sec_batch_sig_std_msig',
  'one_sec_batch_sig_max_msig',
  'one_sec_batch_sig_min_msig',
  'on

In [0]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 677.11 MB (74.2 % reduction)
Mem. usage decreased to 268.94 MB (74.0 % reduction)


In [0]:
_, valid_X, _, valid_y = train_test_split(train[use_cols],train['open_channels'],test_size=0.1)
print(f'x_train shape => {valid_X.shape}, y_val shape => {valid_y.shape}')

# Models

## Tree Based Models

### LightGBM Algorithm

In [0]:
%%capture
params = {'learning_rate': 0.098, 
          'max_depth': -1, 
          'num_leaves': 530,
          'metric': 'rmse', 
          'random_state': 49, 
          'n_jobs':-1, 
          'feature_fraction': 0.8,
          'bagging_fraction': 0.7,
          'bagging_freq': 17,
          'sample_fraction':0.89}

gc.collect()

In [0]:
train_groups = {
    0:[(0,1000000)],
    1:[(1500000,2000000),(3500000,4000000)],
    2:[(2500000,3000000),(4000000,4500000)],
    3:[(1000000,1500000),(3000000,3500000)],
    4:[(2000000,2500000),(4500000,5000000)]
}
train['group_seg'] = -1
for k,v in train_groups.items():
    for l,r in v:
        train.iloc[l:r,-1] = k


In [0]:
#Custom Validation Data Split
folds_valid = []
for x in np.sort(train['group_seg'].unique()):
    folds_valid.append(train[train['group_seg'] == x].index.tolist())
        
folds_train = []
for x in  range(len(folds_valid)):
    folds_train.append(train.loc[~train.index.isin(folds_valid[x])].index.tolist())

In [0]:
predictions_lgb = 0
fold_score = []
for i, (train_idx, val_idx) in enumerate(zip(folds_train,folds_valid)):
  print(f'On fold {i}')
  
  x_train, x_val = train.iloc[train_idx][use_cols], train.iloc[val_idx][use_cols]
  y_train, y_val = train.iloc[train_idx]['open_channels'], train.iloc[val_idx]['open_channels']

  train_data = lgb.Dataset(x_train,label = y_train)
  val_data = lgb.Dataset(x_val,label = y_val)

  model = lgb.train(params, train_data, 2000,  val_data, 
                    verbose_eval=50, early_stopping_rounds=500, feval=MacroF1Metric)
  
  fold_score.append(f1_score(valid_y, np.round(np.clip(model.predict(valid_X),0,10)),average='macro'))
  print(f"F1 Score for fold {i} : {fold_score[-1]}")

  preds = model.predict(test[use_cols],num_iteration=model.best_iteration)
  predictions_lgb += preds

  del x_train,x_val,y_train,y_val,train_data,val_data
  gc.collect()
print(f'Mean Validation Score : {np.mean(fold_score)}')
predictions_lgb /= 5

In [0]:
sub_1 = np.round(np.clip(predictions_lgb,0,10)).astype(int)
pd.Series(sub_1).value_counts()

In [0]:
test['open_channels'] = sub_1
show(test,'signal')

## Deep Learning Models

### Preprocessing

In [0]:
use_cols = [x for x in train.columns if x not in ['time', 'open_channels','batch', 'batch_slices', 'group','mini_batch']]
norm_cols = [x for x in use_cols if x not in ['open_channels','has_10_states']]
use_cols,len(use_cols)

In [0]:
SCALE = True
if SCALE:
  train = std_sc(train,norm_cols)
  test = std_sc(test,norm_cols)

In [0]:
train['group'] = train.groupby(train.index//4000).agg('ngroup').astype(np.int32)
test['group'] = test.groupby(test.index//4000).agg('ngroup').astype(np.int32)
group = train['group']
kf = GroupKFold(n_splits=5)
splits = [x for x in kf.split(train, train['open_channels'], group)]
new_splits = []
for sp in splits:
    new_split = []
    new_split.append(np.unique(group[sp[0]]))
    new_split.append(np.unique(group[sp[1]]))
    new_split.append(sp[1])    
    new_splits.append(new_split)

In [0]:
train_data = np.array(train[use_cols]).reshape((-1,4000,train[use_cols].shape[1]))
test_data = np.array(test[use_cols]).reshape((-1,4000,train[use_cols].shape[1]))
train_target = np.array(pd.get_dummies(train['open_channels'])).reshape((-1,4000,11))
train_data.shape, train_target.shape

### WaveNet Convolution

In [0]:
#Enabling NVIDIA TESLA-P100 GPU Connection
K.clear_session()
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=10,verbose=1)
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.65,patience=10,verbose=1)

In [0]:
LR = 0.001
cb_lr_schedule = LearningRateScheduler(lr_schedule)

In [0]:
aug = False
seed_everything(42)
oof_ = np.zeros((len(train), 11)) 
preds_ = np.zeros((len(test), 11))
for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(new_splits[0:], start=0):
  print(f'On Fold :{n_fold}')
  train_x, train_y = train_data[tr_idx], train_target[tr_idx]
  valid_x, valid_y = train_data[val_idx], train_target[val_idx]
  print(f'Our training dataset shape is {train_x.shape}')
  print(f'Our validation dataset shape is {valid_x.shape}')

  shape_ = (None, train_x.shape[2])
  model = Classifier(shape_)
  model.fit(train_x,train_y,
            epochs=100,
            callbacks=[cb_lr_schedule,MacroF1(model,valid_x,valid_y)],
            batch_size=4,
            validation_data=(valid_x,valid_y))
  

  preds_f = model.predict(valid_x)
  f1_score_ = f1_score(np.argmax(valid_y, axis=2).reshape(-1),  np.argmax(preds_f, axis=2).reshape(-1), average = 'macro')
  print(f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}')
  preds_f = preds_f.reshape(-1, preds_f.shape[-1])
  oof_[val_orig_idx,:] += preds_f
  te_preds = model.predict(test_data)
  te_preds = te_preds.reshape(-1, te_preds.shape[-1])
  preds_ += te_preds
  preds_ = preds_ / 5
f1_score_ = f1_score(np.argmax(train_target, axis = 2).reshape(-1),  np.argmax(oof_, axis = 1), average = 'macro') 
print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')

In [0]:
sub_2 = np.argmax(preds_, axis = 1).astype(int)
pd.Series(sub_2).value_counts()

# Submtting the Predictions

In [0]:
#Ensembleming 2 Models(LightGBM and WaveNet Predictions)
submission['open_channels'] = (sub_1 + sub_2)/2.0
submission['open_channels'] = submission['open_channels'].astype(int)

In [0]:
submission.to_csv('submission.csv', index=False, float_format='%.4f')